In [8]:
import pandas as pd
import numpy as np
import os

In [4]:
non_bot = ["E13.csv"]
bot = ['FSF.csv', 'INT.csv', 'TFP.csv', 'TWT.csv']

In [5]:
for folder in non_bot:
    data = "clean_merged" + ".csv"
    path = os.path.join("./", folder, "clean", data)

In [6]:
df = pd.read_csv(path)

In [7]:
df.sample(10)

,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,num_tweets,retweet_ratio,reply_ration,id,statuses_count,followers_count,friends_count,favourites_count,listed_count,account_age_years,followers_to_friends_ratio,bot
860,418140622,0.000607,0.0,0.007524,0.097870,0.180092,0.052990,0.341598,0.000144,0.0,418140622,0.013346,0.000335,0.011744,0.001082,0.002688,0.333333,0.000434,0
587,291314239,0.000592,0.0,0.000120,0.012486,0.002144,0.009148,0.038567,0.001221,0.0,291314239,0.001578,0.000020,0.006107,0.000023,0.000000,0.333333,0.000049,0
459,210459173,0.069878,0.0,0.022270,0.269923,0.112404,0.193217,0.935109,0.006077,0.0,210459173,0.245602,0.002654,0.070383,0.000857,0.034946,0.500000,0.000573,0
253,93259719,0.002069,0.0,0.026484,0.329600,0.159571,0.353414,0.974901,0.000173,0.0,93259719,0.113355,0.001234,0.031394,0.087736,0.012097,0.666667,0.000597,0
488,226964193,0.099067,0.0,0.011557,0.138817,0.230628,0.196564,0.972452,0.008285,0.0,226964193,0.206415,0.000637,0.027402,0.033642,0.002688,0.500000,0.000353,0
297,114541436,0.000421,0.0,0.005417,0.060595,0.007657,0.033133,0.106520,0.000319,0.0,114541436,0.004470,0.000198,0.014249,0.001759,0.000000,0.500000,0.000211,0
495,231058132,0.004571,0.0,0.000361,0.026809,0.033078,0.033690,0.083563,0.004404,0.0,231058132,0.002329,0.000066,0.004228,0.000383,0.000000,0.500000,0.000238,0
1310,856270100,0.000124,0.0,0.000060,0.009181,0.016539,0.002566,0.030609,0.000320,0.0,856270100,0.001290,0.000007,0.000783,0.000000,0.000000,0.166667,0.000143,0
1410,1037178583,0.001419,0.0,0.000181,0.011017,0.008882,0.010598,0.037343,0.003019,0.0,1037178583,0.001628,0.000091,0.008455,0.000631,0.000000,0.166667,0.000163,0
540,259680844,0.000356,0.0,0.000120,0.029196,0.013170,0.039380,0.064585,0.000443,0.0,259680844,0.002391,0.000113,0.008377,0.000135,0.002688,0.333333,0.000204,0


In [50]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

class ImportData:
    def __init__(self):
        self.non_bot_folder = ["E13.csv", 'TFP.csv']
        self.bot_folder = ['FSF.csv', 'INT.csv', 'TWT.csv']


    def type_data(self, type_data_merged):
        # Check the type of data 
        if type_data_merged == 1:
            return "clean_merged.csv"
        else:
            return "clean_users.csv"

    def read_and_sample_data(self, base_path="../Data/cresci-2015.csv/", type_data_merged = 1, bot_ratio=[.2, .8], bot_fldr_ratio=[1, 1, 1]):
        bot_data_frames = pd.DataFrame()
        total_bot = 0

        # Check the type of data to use
        type_data = self.type_data(type_data_merged)

        # Read non-bot data
        for folder in self.non_bot_folder:
            # Read path
            file_path = os.path.join(base_path, folder, "clean", type_data)
            if os.path.exists(file_path):
                non_bot_data_frames = pd.read_csv(file_path)
                total_non_bot = len(non_bot_data_frames)
                
        # Read and sample bot data
        for idx, folder in enumerate(self.bot_folder):
            file_path = os.path.join(base_path, folder, "clean", type_data)
            if os.path.exists(file_path):
                if bot_fldr_ratio[idx] == 1:
                    df = pd.read_csv(file_path)
                    bot_data_frames = pd.concat([bot_data_frames,  df], ignore_index=True)
                total_bot = len(bot_data_frames)

        # Calculate available ratios
        total_samples = total_bot + total_non_bot
        required_bot_samples = total_samples * bot_ratio[1]
        required_non_bot_samples = total_samples * bot_ratio[0]

        # Determine how many rows to keep based on the desired ratio and availability
        if required_bot_samples > total_bot:  # If less bot data is available than desired
            # Adjust bot rows to match the actual bot ratio
            required_bot_samples = total_bot
            required_non_bot_samples = round((bot_ratio[0] * required_bot_samples) / bot_ratio[1])

        elif required_non_bot_samples > total_non_bot:  # If less bot data is available than desired
            # Adjust non-bot rows to match the actual bot ratio
            required_non_bot_samples = total_non_bot
            required_bot_samples = round((bot_ratio[1] * required_non_bot_samples) / bot_ratio[0])

        # Sample 
        non_bot_df = non_bot_data_frames.sample(n=required_non_bot_samples)
        bot_df = bot_data_frames.sample(n=required_bot_samples)

        # Merge the adjusted data
        final_df = pd.concat([non_bot_df, bot_df], ignore_index=True)
        return final_df
    
    def split_dataset(self, data, proportions = [.7,.15,.15], target='bot'):
        """
        Splits the dataset into training, testing, and validation sets based on the provided proportions.
        
        Parameters:
        - data: The DataFrame to split.
        - proportions: A list of three numbers representing the proportion of training, testing, and validation sets, respectively.
        - target: The name of the target feature.
        
        Returns:
        A dictionary with keys 'X_train', 'X_test', 'X_val', 'y_train', 'y_test', 'y_val' and their corresponding DataFrame or Series as values.
        """
        
        if sum(proportions) != 1:
            raise ValueError("Proportions must sum up to 1.")
        
        train_prop, test_prop, val_prop = proportions
        
        # Split data into training and temporary data (temp will be split into test and validation)
        train_data, temp_data, y_train, y_temp = train_test_split(
            data.drop(target, axis=1), data[target], test_size=(1-train_prop), stratify=data[target], random_state=42)
        
        # Calculate proportion of test set relative to temp data
        test_relative_prop = test_prop / (test_prop + val_prop)
        
        # Split temp data into testing and validation
        X_test, X_val, y_test, y_val = train_test_split(
            temp_data, y_temp, test_size=(1-test_relative_prop), stratify=y_temp, random_state=42)
        
        # Prepare the result dictionary
        result = {
            'X_train': train_data,
            'X_test': X_test,
            'X_val': X_val,
            'y_train': y_train,
            'y_test': y_test,
            'y_val': y_val
        }
        
        return result
    

In [53]:
d = ImportData()
data = d.read_and_sample_data(bot_ratio=[.4, .6])
data['bot'].value_counts(normalize=True)

1    0.600172
0    0.399828
Name: bot, dtype: float64

In [58]:
splits = d.split_dataset(data)
splits['y_train'].value_counts(normalize=True)

1    0.600737
0    0.399263
Name: bot, dtype: float64

In [59]:
splits['y_test'].value_counts(normalize=True)

1    0.597701
0    0.402299
Name: bot, dtype: float64

In [60]:
splits['X_train']

,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,num_tweets,retweet_ratio,reply_ration,statuses_count,followers_count,friends_count,favourites_count,listed_count,account_age_years,followers_to_friends_ratio
331,96727791,3.524257e-02,0.015529,0.000153,0.018680,0.012341,0.029877,0.064516,1.312559e-02,0.188172,0.003865,0.000724,0.002814,0.000194,0.001316,0.571429,0.000937
237,393255100,8.269085e-01,0.010353,0.000000,0.016089,0.023272,0.031389,0.072736,2.732417e-01,0.111302,0.004221,0.001753,0.003738,0.000065,0.001316,0.285714,0.001714
409,429414906,2.250420e-03,0.000324,0.000765,0.133488,0.188470,0.076561,0.260546,2.079138e-04,0.000972,0.015764,0.019591,0.049345,0.001549,0.009211,0.285714,0.001466
762,616298898,1.831070e-03,0.154955,0.000489,0.017070,0.004119,0.005497,0.083762,1.651718e-02,0.206146,0.072970,0.369863,0.481481,0.000000,0.000000,0.250000,0.175416
685,48638750,7.467769e-07,0.000000,0.000006,0.000000,0.018300,0.000733,0.034595,2.461415e-07,0.000000,0.003562,0.036946,0.163171,0.000000,0.093750,0.666667,0.052608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,626820204,0.000000e+00,0.000000,0.000000,0.000000,0.000282,0.000367,0.000591,0.000000e+00,0.000000,0.000091,0.006773,0.248593,0.000000,0.000000,0.166667,0.006634
587,1174998998,0.000000e+00,0.003636,0.000000,0.103448,0.000000,0.000000,0.384615,0.000000e+00,0.004211,0.542857,0.407407,0.717687,0.000000,0.000000,0.000000,0.424785
1032,617695084,0.000000e+00,0.003604,0.000000,0.002845,0.000000,0.000000,0.019104,0.000000e+00,0.020419,0.017132,0.328767,0.147147,0.000000,0.000000,0.250000,0.510204
768,1175833148,4.071063e-01,0.000000,0.930004,0.275862,0.250000,0.153846,0.500000,4.256112e-01,0.000000,0.628571,0.259259,0.748299,0.000000,0.000000,0.000000,0.259259


In [61]:
data

,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,num_tweets,retweet_ratio,reply_ration,statuses_count,followers_count,friends_count,favourites_count,listed_count,account_age_years,followers_to_friends_ratio,bot
0,154547583,3.206141e-02,0.000647,0.000765,0.140305,0.090092,0.083230,0.227202,0.003397,0.002230,0.012762,0.018600,0.045943,0.000323,0.007895,0.428571,0.001495,0
1,39588706,3.679271e-04,0.000000,0.000000,0.000000,0.000000,0.000178,0.000310,0.019047,0.000000,0.000045,0.000114,0.000336,0.000129,0.000000,0.571429,0.001118,0
2,1017001452,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000089,0.000000,0.000000,0.000000,0.000009,0.000114,0.000630,0.000000,0.000000,0.142857,0.000629,0
3,85850428,0.000000e+00,0.000971,0.000000,0.000273,0.000176,0.000356,0.000931,0.000000,0.700605,0.000062,0.000191,0.002436,0.000000,0.000000,0.571429,0.000284,0
4,515405009,9.639690e-03,0.001294,0.000612,0.021134,0.060296,0.038058,0.069634,0.003327,0.014531,0.004088,0.001372,0.003318,0.003486,0.001316,0.142857,0.001509,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,619445404,1.052097e-07,0.001802,0.000001,0.008535,0.000000,0.000000,0.019838,0.000004,0.009845,0.017766,0.191781,0.140641,0.000000,0.000000,0.250000,0.311388,1
1159,1174933712,0.000000e+00,0.000000,0.000000,0.068966,0.000000,0.000000,0.115385,0.000000,0.000000,0.342857,0.370370,0.697279,0.000000,0.000000,0.000000,0.397471,1
1160,1175793452,0.000000e+00,0.000000,0.000000,0.068966,0.000000,0.000000,0.346154,0.000000,0.000000,0.514286,0.370370,0.687075,0.000000,0.000000,0.000000,0.403374,1
1161,1174723886,0.000000e+00,0.000000,0.000000,0.068966,0.083333,0.076923,0.269231,0.000000,0.000000,0.457143,0.185185,0.710884,0.000000,0.000000,0.000000,0.194932,1
